In [1]:
import pandas as pd

# Read Metadata
meta_df = pd.read_csv('/Users/brensikeg/Desktop/Final/archive/fer2013/fer2013.csv')

emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

meta_df = meta_df.rename(columns={"emotion":"label","pixels":"image"})
meta_df['emotion'] = meta_df['label'].apply(lambda x: emotions[int(x)])
meta_df = meta_df.drop(columns=['Usage'])

In [2]:
import numpy as np

meta_df["new_image"] = meta_df["image"].apply(lambda x: np.array(np.reshape(np.array(x.split(' ')).astype(int),(48,48))))
meta_df = meta_df.drop(columns=['image'])

In [3]:
meta_df

,label,emotion,new_image
0,0,Angry,"[[70, 80, 82, 72, 58, 58, 60, 63, 54, 58, 60, ..."
1,0,Angry,"[[151, 150, 147, 155, 148, 133, 111, 140, 170,..."
2,2,Fear,"[[231, 212, 156, 164, 174, 138, 161, 173, 182,..."
3,4,Sad,"[[24, 32, 36, 30, 32, 23, 19, 20, 30, 41, 21, ..."
4,6,Neutral,"[[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 15, 2..."
...,...,...,...
35882,6,Neutral,"[[50, 36, 17, 22, 23, 29, 33, 39, 34, 37, 37, ..."
35883,3,Happy,"[[178, 174, 172, 173, 181, 188, 191, 194, 196,..."
35884,0,Angry,"[[17, 17, 16, 23, 28, 22, 19, 17, 25, 26, 20, ..."
35885,3,Happy,"[[30, 28, 28, 29, 31, 30, 42, 68, 79, 81, 77, ..."


In [4]:
import dlib
import cv2
from imutils import face_utils

hogFaceDetector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/Users/brensikeg/Desktop/Final/shape_predictor_68_face_landmarks.dat")

features = []

for image in meta_df["new_image"]:
    gray = image.astype('uint8')
    faces = hogFaceDetector(gray, 1)
    
    for (i, rect) in enumerate(faces):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        
    features.append(shape)

In [5]:
import math

new_features = []
for row in features:
    means = row.mean(axis=0)

    tmp = []
    
    for pt in row:
        mag = math.sqrt((means[0]-pt[0])*(means[0]-pt[0]) + (means[1]-pt[1])*(means[1]-pt[1]))
        bearing = math.atan((means[1]-pt[1])/(means[0]-pt[0]))
        
        tmp.append([pt[0], pt[1], mag, bearing])
    
    tmp = np.concatenate(tmp, axis=0)
    
    new_features.append(tmp)
    

<ipython-input-5-cb369852f8b3>:11: RuntimeWarning: divide by zero encountered in double_scalars
  bearing = math.atan((means[1]-pt[1])/(means[0]-pt[0]))


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.array(new_features), np.array(meta_df["label"]), test_size=0.20, random_state=42)

In [7]:
from sklearn.svm import SVC

clf = SVC(kernel='linear', class_weight='balanced')
clf = clf.fit(X_train, y_train)

In [8]:
print("Predicting emotions on the testing set")
y_pred = clf.predict(X_test)

Predicting emotions on the testing set


In [9]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report(y_test, y_pred, target_names=emotions))

              precision    recall  f1-score   support

       Angry       0.33      0.32      0.33       985
     Disgust       0.05      0.40      0.09       102
        Fear       0.24      0.13      0.17      1043
       Happy       0.63      0.59      0.61      1765
         Sad       0.31      0.20      0.24      1210
    Surprise       0.44      0.53      0.48       795
     Neutral       0.36      0.41      0.38      1278

    accuracy                           0.38      7178
   macro avg       0.34      0.37      0.33      7178
weighted avg       0.40      0.38      0.38      7178



In [10]:
print(confusion_matrix(y_test, y_pred, labels=range(len(emotions))))

[[ 316  138   76   94   99   82  180]
 [  24   41    5    6   10    5   11]
 [ 129  172  136  136  136  160  174]
 [ 106  154  104 1034   88  107  172]
 [ 165  140  107  168  241   97  292]
 [  69   51   52   92   34  423   74]
 [ 145  155   90  122  158   90  518]]


In [11]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, y_pred)
print(score)

0.3774031763722485
